#In this file, we create two feature groups on hopsworks.


*   Reading two datasets from our github url
*   clone the git hub repository here, so we have access easily to "functions.py" 
*   transforming some columns like date by functions.py's methods
*   create two feature groups and saving them on hopsworks


In [ ]:
!pip install -U hopsworks 

In [43]:
import pandas as pd
import sys

In [44]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Clone our shared **github repository** to be able to import functions.py file

In [45]:
!git clone 'https://github.com/reyrobs/Scalable_ML_Project_AQ.git'

fatal: destination path 'Scalable_ML_Project_AQ' already exists and is not an empty directory.


In [46]:
from functions import *

###Reading air data set(pm10, pm...) from Github:

In [47]:
url = 'https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/Datasets/air_quality_df.csv'
df_air_quality = pd.read_csv(url)

df_air_quality.head()

,date,pm25,pm10,o3,no2,so2,co,AQI
0,2022/12/1,78.0,37.0,4.0,18.0,0.0,0.0,78.0
1,2022/12/2,93.0,24.0,3.0,17.0,0.0,0.0,93.0
2,2022/12/3,68.0,30.0,5.0,17.0,0.0,0.0,68.0
3,2022/12/4,79.0,29.0,1.0,16.0,0.0,0.0,79.0
4,2022/12/5,77.0,34.0,0.0,17.0,0.0,0.0,77.0


In [48]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time_slash)
#df_air_quality.sort_values(by = ['city','date'],inplace = True,ignore_index = True)
df_air_quality.sort_values(by = ['date'],inplace = True,ignore_index = True)
df_air_quality.head()

,date,pm25,pm10,o3,no2,so2,co,AQI
0,1388448000000,0.0,19.0,22.0,30.0,0.0,2.0,30.0
1,1388534400000,43.0,30.0,22.0,42.0,0.0,4.0,43.0
2,1388620800000,53.0,33.0,22.0,34.0,0.0,2.0,53.0
3,1388707200000,50.0,28.0,23.0,41.0,0.0,3.0,50.0
4,1388793600000,52.0,23.0,19.0,34.0,0.0,3.0,52.0


###Reading weather data(winddir,...)  from Github 

In [49]:
url = 'https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/Datasets/visual_crossing_df.csv'
df_weather = pd.read_csv(url)

df_weather.head(3)

,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,moonphase
0,2022-01-01,13.9,7.8,10.5,13.9,6.8,10.0,8.5,88.2,0.048,...,0.0,25.3,13.0,178.8,1024.6,77.6,14.5,33.6,2.9,0.99
1,2022-01-02,14.0,9.5,11.9,14.0,8.4,11.7,8.2,78.5,0.144,...,0.0,53.6,24.5,221.0,1018.9,89.8,24.3,18.2,1.5,1.00
2,2022-01-03,12.7,10.8,11.5,12.7,10.8,11.5,9.1,85.5,8.867,...,0.0,52.3,22.8,224.0,1015.4,97.9,17.0,18.9,1.7,0.00


In [50]:
df_weather.shape

(354, 23)

In [51]:
df_weather.date = df_weather.date.apply(timestamp_2_time_dash)
df_weather.sort_values(by=['date'],inplace=True, ignore_index=True)
df_weather.head(3)

,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,moonphase
0,1640995200000,13.9,7.8,10.5,13.9,6.8,10.0,8.5,88.2,0.048,...,0.0,25.3,13.0,178.8,1024.6,77.6,14.5,33.6,2.9,0.99
1,1641081600000,14.0,9.5,11.9,14.0,8.4,11.7,8.2,78.5,0.144,...,0.0,53.6,24.5,221.0,1018.9,89.8,24.3,18.2,1.5,1.00
2,1641168000000,12.7,10.8,11.5,12.7,10.8,11.5,9.1,85.5,8.867,...,0.0,52.3,22.8,224.0,1015.4,97.9,17.0,18.9,1.7,0.00


In [52]:
#!pip install pandas
import hopsworks

In [53]:
project = hopsworks.login()

fs = project.get_feature_store() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5315
Connected. Call `.close()` to terminate connection gracefully.


In [55]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality_fg',
        description = 'Air Quality characteristics of each day',
        version = 1,
        primary_key = ['date'],# ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    
air_quality_fg.insert(df_air_quality)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5315/fs/5235/fg/13682


Uploading Dataframe: 0.00% |          | Rows 0/3146 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/air_quality_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f054d5ffc10>, None)

In [56]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5315/fs/5235/fg/13683


Uploading Dataframe: 0.00% |          | Rows 0/354 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f056d316910>, None)